# Limpieza de datos y agregaciones

### OBJETIVO

- Limpiar nuestros datasets de `NaNs`.
- Reindexar si es necesario
- Renombrar columnas si es necesario
- Experimentar la aplicación de agregaciones para explorar nuestro dataset

## Carga general de datos

Para nuestra limpieza de datos, solo utilizaremos las siguientes librerias:

In [1]:
import pandas as pd
import numpy as np

Vamos a realizar lo siguiente:

- Cargar los documentos que limpiaremos, con el objetivo de dejar un par de archivos `csv` al final
- Concentrar los datos por tipo de PM

In [2]:
df_PM10_2019 = pd.read_csv('../Datasets/PM10/2019PM10.csv') #Data PM 10 - 2019
df_PM10_2020 = pd.read_csv('../Datasets/PM10/2020PM10.csv') #Data PM 10 - 2020
df_PM25_2019 = pd.read_csv('../Datasets/PM2.5/2019PM25.csv') #Data PM 2.5 - 2019
df_PM25_2020 = pd.read_csv('../Datasets/PM2.5/2020PM25.csv') #Data PM 2.5 - 2020

Después de cargar nuestros archivos `csv`, los concatenamos para tener dos Dataframes (df), con los cuales vamos a realizar nuestra depuración de datos.

In [3]:
df_PM10_total = pd.concat([df_PM10_2019 , df_PM10_2020], axis=0)
df_PM25_total = pd.concat([df_PM25_2019 , df_PM25_2020], axis=0)

## Depuración de archivos $PM_{10}$

Comenzamos depurando la información de $PM_{10}$  
De acuerdo a las especificaciones RAMA, los datos nulos se identifican con la etiqueta -99, por lo que, convertiremos los valores `-99` en `NaN`. 

In [4]:
df_PM10_total = df_PM10_total.replace(-99, np.nan)

Haciendo una rápida validación de nuestro df que concentra los datos de $PM_{10}$, vemos que contiene **17,544** `filas` y **29** `columnas`.

In [5]:
df_PM10_total.shape

(17544, 29)

Ahora hacemos una revisión de los datos nulos que contiene nuestro df.   
Podemos ver, que todas las filas y las columnas de las estaciones cuentan con al menos un valor `NaN`

In [6]:
# Filas con NaN
df_PM10_total.isna().any(1).sum()

17544

In [7]:
# Columnas con NaN
df_PM10_total.isna().any()

FECHA    False
HORA     False
ACO       True
AJM       True
ATI       True
BJU       True
CAM       True
CHO       True
CUA       True
CUT       True
FAC       True
FAR       True
GAM       True
HGM       True
INN       True
IZT       True
MER       True
MGH       True
MPA       True
PED       True
SAC       True
SAG       True
SFE       True
TAH       True
TLA       True
TLI       True
UIZ       True
VIF       True
XAL       True
dtype: bool

Revisando la parte proporcional de `NaNs` por cada columna, observamos que la mayoría de nuestras estaciones cuentan con un alto porcentaje de datos nulos.
Y que la media de nuestras medias del df corresponde al **39.76%** de `NaN`. Una cifra bastante alta, por cierto.

In [8]:
df_PM10_total.isna().mean()

FECHA    0.000000
HORA     0.000000
ACO      0.501482
AJM      0.435192
ATI      0.187699
BJU      0.351231
CAM      0.378819
CHO      0.892613
CUA      0.359211
CUT      0.115367
FAC      0.154925
FAR      1.000000
GAM      0.298792
HGM      0.549248
INN      0.342681
IZT      0.234553
MER      0.156692
MGH      0.813383
MPA      1.000000
PED      0.109325
SAC      1.000000
SAG      0.428409
SFE      0.120383
TAH      0.339261
TLA      0.222070
TLI      0.615424
UIZ      0.205882
VIF      0.194140
XAL      0.525992
dtype: float64

In [9]:
df_PM10_total.isna().mean().mean() * 100

39.76818875104171

Considerando lo anterior, vamos a eliminar aquellas columnas que contengan al menos el **55%** de valores `NaNs`  
Con esto, nos estamos quedando ahora con **23** columnas en total.

In [10]:
# Se eliminan con el 55% de NaN
df_PM10_depurado = df_PM10_total.dropna(thresh=len(df_PM10_total)*0.45, axis=1)

In [11]:
df_PM10_depurado.shape

(17544, 23)

A pesar de que eliminamos columnas con un alto porcentaje de nulos, al validar nuestro df nuevamente, nos percatamos que todas nuestras filas tienen al menos un valor `NaN`, y ninguna fila con todos los valores `NaN`. Por lo tanto, no podemos eliminar las filas, debido a que nos estaríamos quedando sin datos para procesar.  
Tampoco es posible, por esta estructura que tienen nuestros df, reemplazar estos valores `NaN` con 0, debido a que la naturaleza de nuestros análisis requieren obtener medias, y este valor nos estaría dando una tendencia errónea en nuestros resultados.

In [12]:
df_PM10_depurado[df_PM10_depurado.isna().any(axis=1)].shape[0]

17544

In [13]:
df_PM10_depurado.dropna(axis=0, how='all').shape

(17544, 23)

Realizamos un ejemplo de agregado (mean), que se puede aplicar a nuestro df depurado

In [14]:
df_PM10_depurado.mean()

HORA    12.500000
ACO     47.815573
AJM     33.896761
ATI     38.657568
BJU     32.202337
CAM     42.013030
CUA     32.840954
CUT     46.794459
FAC     34.396735
GAM     43.321980
HGM     41.718766
INN     23.570239
IZT     36.863728
MER     46.755728
PED     31.153718
SAG     53.723175
SFE     31.337740
TAH     49.991373
TLA     47.288247
UIZ     41.892621
VIF     49.135097
XAL     67.097763
dtype: float64

Por último, guardamos nuestro df depurado en un nuevo `csv` de $PM_{10}$

In [15]:
#Guardar csv
df_PM10_depurado.to_csv('../Datasets/PM10/PM10_depurado.csv')

## Depuración de archivos $PM_{2.5}$

Ahora depuramos la información de $PM_{2.5}$  
De acuerdo a las especificaciones RAMA, los datos nulos se identifican con la etiqueta -99, por lo que, convertiremos los valores `-99` en `NaN`. 

In [16]:
df_PM25_total = df_PM25_total.replace(-99, np.nan)

Haciendo una rápida validación de nuestro df que concentra los datos de $PM_{2.5}$, vemos que contiene **16,800** `filas` y **26** `columnas`.

In [17]:
#Obtenemos las columnas
df_PM25_total.shape

(16800, 26)

Aplicamos la revisión de los datos nulos que contiene nuestro df.  
Podemos ver que en este df, también todas las filas y las columnas de las estaciones cuentan con al menos un valor `NaN`

In [18]:
# Filas con NaN
df_PM25_total.isna().any(1).sum()

16800

In [19]:
# Columnas con NaN
df_PM25_total.isna().any()

FECHA    False
HORA     False
AJM       True
AJU       True
BJU       True
CAM       True
CCA       True
COY       True
FAR       True
GAM       True
HGM       True
INN       True
MER       True
MGH       True
MON       True
MPA       True
NEZ       True
PED       True
SAC       True
SAG       True
SFE       True
SJA       True
TLA       True
UAX       True
UIZ       True
XAL       True
dtype: bool

Revisando la parte proporcional de `NaNs` por cada columna, observamos que la mayoría de nuestras estaciones también cuentan con un alto porcentaje de datos nulos.
En este caso, la media de nuestras medias del df corresponde al **39.69%** de `NaN`. Una cifra igual de alta, por cierto.

In [20]:
df_PM25_total.isna().mean()

FECHA    0.000000
HORA     0.000000
AJM      0.410179
AJU      0.538036
BJU      0.366667
CAM      0.361071
CCA      0.129524
COY      1.000000
FAR      0.149226
GAM      0.311548
HGM      0.529286
INN      0.356250
MER      0.161548
MGH      0.805119
MON      0.419345
MPA      1.000000
NEZ      0.209464
PED      0.113393
SAC      0.212500
SAG      0.445952
SFE      0.124643
SJA      1.000000
TLA      0.229643
UAX      0.213512
UIZ      0.208452
XAL      0.505000
dtype: float64

In [21]:
df_PM25_total.isna().mean().mean() * 100

37.69368131868132

Al igual que en nuestro df anterior, vamos a eliminar aquellas columnas que contengan al menos el **55%** de valores `NaNs`  
Con esto, nos estamos quedando ahora con **22** columnas en total.

In [22]:
# Se eliminan con el 55% de NaN

df_PM25_depurado = df_PM25_total.dropna(thresh=len(df_PM25_total)*0.45, axis=1)

In [23]:
df_PM25_depurado.shape

(16800, 22)

En este caso, nos percatamos que salvo 2 filas, en este df las filas contienen al menos un valor `NaN`, y ninguna fila con todos los valores `NaN`. Por lo tanto, no podemos eliminar las filas, debido a que nos estaríamos quedando sin datos para procesar.  
Tampoco es posible, por esta estructura que tienen nuestros df, reemplazar estos valores `NaN` con 0, debido a que la naturaleza de nuestros análisis requieren obtener medias, y este valor nos estaría dando una tendencia errónea en nuestros resultado.

In [24]:
df_PM25_depurado[df_PM25_depurado.isna().any(axis=1)].shape

(16798, 22)

Realizamos un ejemplo de agregado (mean), que se puede aplicar a nuestro df depurado

In [25]:
df_PM25_depurado.mean()

HORA    12.500000
AJM     19.096983
AJU     18.674655
BJU     18.110902
CAM     22.833799
CCA     19.181961
FAR     18.966347
GAM     22.199377
HGM     24.675013
INN     13.949145
MER     23.063041
MON     20.194567
NEZ     24.976282
PED     18.365760
SAC     24.307181
SAG     22.954340
SFE     17.312593
TLA     23.082367
UAX     19.571634
UIZ     21.539555
XAL     26.067581
dtype: float64

Por último, guardamos nuestro df depurado en un nuevo `csv` de $PM_{2.5}$

In [26]:
#Guardar csv
df_PM25_depurado.to_csv('../Datasets/PM2.5/PM25_depurado.csv')

## Depuración de cat_estacion

El siguiente archivo a depurar, es el catálogo de estaciones y zonas.  
Cargamos nuestro archivo, y validamos 

In [27]:
df_cat_estacion = pd.read_csv('../Datasets/cat_estacion.csv') #Data catálogo de estaciones

Haciendo una rápida validación de nuestro df con el catálogo de estaciones y zonas, vemos que contiene **69** `filas` y **8** `columnas`.

In [28]:
df_cat_estacion.shape

(69, 8)

Verificamos, cuántas filas y columnas cuentan con datos `NaN`  
Confirmamos que en este df no tenemos tantos nulos, por lo que borraremos laa columnas que contengan datos `NaN` y que no hacen inferencia en nuestro análisis

In [29]:
df_cat_estacion.isna().any(1).sum()

49

In [30]:
df_cat_estacion.isna().any()

cve_estac     False
nom_estac     False
longitud      False
latitud       False
alt            True
obs_estac      True
id_station    False
Zona          False
dtype: bool

In [31]:
df_cat_estacion.isna().mean()

cve_estac     0.000000
nom_estac     0.000000
longitud      0.000000
latitud       0.000000
alt           0.014493
obs_estac     0.710145
id_station    0.000000
Zona          0.000000
dtype: float64

In [32]:
df_cat_estacion = df_cat_estacion.drop(['longitud','latitud','alt','obs_estac','id_station'], axis=1)

Al final, después de depurar, solo nos quedamos con **3** columnas y ningún valor `NaN`

In [33]:
df_cat_estacion.isna().mean()

cve_estac    0.0
nom_estac    0.0
Zona         0.0
dtype: float64

Guardamos nuestro nuevo catalogo de estaciones depurado en otro `csv`

In [34]:
df_cat_estacion.to_csv('../Datasets/cat_estacion_depurado.csv')